# MLFlow DEMO

In [5]:
import mlflow
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("test_experiment")

<Experiment: artifact_location='file:///C:/Users/kacpe/OneDrive/Pulpit/Uczelnia/semestr5/MachineLearning/mlruns/1', creation_time=1704549359680, experiment_id='1', last_update_time=1704549359680, lifecycle_stage='active', name='test_experiment', tags={}>

In [3]:
mlflow.sklearn.autolog(disable=True)

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

with mlflow.start_run(run_name='rf_baseline'):
    params = {
        "n_estimators": 100,
        "max_depth": 20
    }

    mlflow.set_tag("model_name", "RF")
    mlflow.log_params(params)
    
    # Create a Random Forest Classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Train the model
    clf.fit(X_train, y_train)
    
    # Make predictions
    y_pred = clf.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)

    mlflow.log_metric("test_rmse", accuracy)
    mlflow.sklearn.log_model(clf, "sk_models")

In [9]:
logged_model = 'runs:/84b9ee80d8814f4ab3cbf205745c7542/sk_models' # string wzięty z serwera lokalnego

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


In [ ]:
# Example new data points
new_data = np.array([
    [5.1, 3.5, 1.4, 0.2],  # Measurements for first flower
    [6.1, 2.9, 4.7, 1.4]   # Measurements for second flower
])

In [8]:
# Make predictions
predictions = loaded_model.predict(new_data)

# Print predictions
for i, prediction in enumerate(predictions):
    print(f"Flower {i + 1} is predicted to be of species: {iris.target_names[prediction]}")

NameError: name 'model' is not defined

## Przykład funkcji do eksperymentowania z naszymi modelami
W naszym przypadku możemy czegoś w tym stylu używać
Dzięki temu, że będziemy mieli funkcję, to lepiej to będzie wyglądało niż jak będziemy przekopiowywać

In [ ]:
def mlp_mlflow_run(
    name,
    mlp_params,
    train_params,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.log_params(mlp_params)
        mlflow.log_params(train_params)
        mlflow.set_tag("model_name", "MLP")
        mlp = build_mlp(mlp_params)
        mlp = train_mlp(mlp, train_params, train_dataset, val_dataset)
        test_preds = mlp.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds.ravel(), squared=False
        )
        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(mlp, "tf_models")